## LLM Hackathon

In [ ]:
import pandas as pd

# Load the data files
train_data = pd.read_csv('/mnt/data/train.csv')
test_data = pd.read_csv('/mnt/data/test.csv')
train_prompts = pd.read_csv('/mnt/data/train_prompts.csv')
submission_example = pd.read_csv('/mnt/data/submission.csv')

# Display the first few rows of each dataframe to understand their structure
train_data_head = train_data.head()
test_data_head = test_data.head()
train_prompts_head = train_prompts.head()
submission_example_head = submission_example.head()

train_data_head, test_data_head, train_prompts_head, submission_example_head


In [ ]:
# Joining the train and test data with the prompts
train_data_merged = pd.merge(train_data, train_prompts, on='prompt_id', how='left')
test_data_merged = pd.merge(test_data, train_prompts, on='prompt_id', how='left')

# Checking the first few rows of the merged data
train_data_merged_head = train_data_merged.head()
test_data_merged_head = test_data_merged.head()

train_data_merged_head, test_data_merged_head


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

# Feature Engineering
# 1. Text Length
train_data_merged['text_length'] = train_data_merged['text'].apply(len)
test_data_merged['text_length'] = test_data_merged['text'].apply(len)

# 2. Word Count
train_data_merged['word_count'] = train_data_merged['text'].apply(lambda x: len(x.split()))
test_data_merged['word_count'] = test_data_merged['text'].apply(lambda x: len(x.split()))

# 3. Average Word Length
train_data_merged['avg_word_length'] = train_data_merged['text_length'] / train_data_merged['word_count']
test_data_merged['avg_word_length'] = test_data_merged['text_length'] / test_data_merged['word_count']

# Selecting features and target variable for the model
features = ['text_length', 'word_count', 'avg_word_length']
X = train_data_merged[features]
y = train_data_merged['generated']

# Splitting the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

# Model Building: Using a RandomForestClassifier for this task
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions on the validation set
y_pred = model.predict_proba(X_valid)[:, 1]

# Evaluating the model using ROC AUC score
roc_auc = roc_auc_score(y_valid, y_pred)
roc_auc


In [ ]:
# Preparing the test dataset for prediction
X_test = test_data_merged[features]

# Making predictions on the test set
test_predictions = model.predict_proba(X_test)[:, 1]

# Preparing the submission file
submission = pd.DataFrame({'id': test_data_merged['id'], 'generated': test_predictions})

# Ensuring the submission format matches the example provided
submission.head(), submission_example_head


In [ ]:
# Saving the submission file as a CSV without the index
submission_file_path = '/mnt/data/my_submission_file.csv'
submission.to_csv(submission_file_path, index=False)

submission_file_path
